In [1]:
# !pip install -r ../../requirement.txt


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import re
import string
import joblib

In [3]:
df = pd.read_csv('E:/Tài liệu môn học/Đồ án/pbl7/modelAI/ModelAI/data/book_user_ratings_include_user_data.csv')
df

,rating_id,book_id,user_id,rating,age,gender,area
0,0,248957,10527,5,39,Male,european
1,1,248957,6843,5,35,Male,south_asian
2,2,248957,3650,5,58,Male,celtic_english
3,3,248957,11592,5,51,Male,celtic_english
4,4,248957,1205,5,60,Female,celtic_english
...,...,...,...,...,...,...,...
200117,200117,250730,8627,5,39,Female,east_asian
200118,200118,250730,44,5,33,Male,east_asian
200119,200119,250730,9117,5,19,Female,east_asian
200120,200120,250730,10149,5,53,Female,celtic_english


columns = ['id', 'title', 'description', 'book_cover', 'image_url', 'release_date', 'publisher', 'number_of_pages', 'price', 'authors', 'rating', 'number_of_ratings', 'number_of_reviews']

In [4]:
# Sử dụng LabelEncoder từ thư viện scikit-learn
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Mã hóa cột 'area' thành dạng số
df['gender_encoded'] = label_encoder.fit_transform(df['gender'])
df['area_encoded'] = label_encoder.fit_transform(df['area'])


In [5]:
df 

,rating_id,book_id,user_id,rating,age,gender,area,gender_encoded,area_encoded
0,0,248957,10527,5,39,Male,european,1,3
1,1,248957,6843,5,35,Male,south_asian,1,9
2,2,248957,3650,5,58,Male,celtic_english,1,1
3,3,248957,11592,5,51,Male,celtic_english,1,1
4,4,248957,1205,5,60,Female,celtic_english,0,1
...,...,...,...,...,...,...,...,...,...
200117,200117,250730,8627,5,39,Female,east_asian,0,2
200118,200118,250730,44,5,33,Male,east_asian,1,2
200119,200119,250730,9117,5,19,Female,east_asian,0,2
200120,200120,250730,10149,5,53,Female,celtic_english,0,1


In [6]:
df_book = pd.read_csv('E:/PBL6/ModelAI/data/thrift-preprocessed-summarize-problem.csv')
df_book


,Unnamed: 0,id,title,description,book_cover,image_url,release_date,publisher,number_of_pages,price,authors,rating,number_of_ratings,number_of_reviews
0,0,18599856,Where the Crawdads Sing,major motion picture 1 new york time bestselli...,Paperback,https://i.thriftbooks.com/api/imagehandler/m/4...,0001-01-01T00:00:00,Corsair,303,14.39,Delia Owens,3.897698,391,95
1,1,248957,Harry Potter and the Sorcerer's Stone,beloved first book harry potter series fully i...,Paperback,https://i.thriftbooks.com/api/imagehandler/m/9...,1998-09-01T00:00:00,Arthur A. Levine Books,320,4.29,J.K. Rowling,3.405858,239,38
2,2,247716,Nineteen Eighty-Four,written 75 year ago 1984 george orwell chillin...,Paperback,https://i.thriftbooks.com/api/imagehandler/m/0...,0001-01-01T00:00:00,NaN,0,187.89,George Orwell,3.695279,233,24
3,3,248147,To Kill a Mockingbird,voted america best loved novel pb great americ...,Mass Market Paperback,https://img.thriftbooks.com/api/images/m/8594c...,1988-10-11T00:00:00,Grand Central Publishing,288,4.19,Harper Lee,3.548387,186,35
4,4,246270,The Alchemist,international bestseller 80 million copy sold ...,Paperback,https://i.thriftbooks.com/api/imagehandler/m/1...,1993-04-25T00:00:00,HarperOne,197,5.49,Paulo Coelho,4.002950,339,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10045,10045,13866451,The State of Affairs: Rethinking Infidelity,fresh look infidelity broadening focus havoc w...,Paperback,https://i.thriftbooks.com/api/imagehandler/m/B...,2018-10-09T00:00:00,Harper Paperbacks,352,10.49,Esther Perel,3.666667,3,1
10046,10046,245996,Your Four-Year-Old: Wild and Wonderful,four year old make lovable problem four year o...,Paperback,https://m.media-amazon.com/images/I/41XFP1mCqL...,1980-09-15T00:00:00,Delta,160,6.99,"Louise Bates Ames,Frances L. Ilg",5.000000,5,5
10047,10047,376019,A Hunger for God: Desiring God through Fasting...,encourage read book asking great thing god. da...,Paperback,https://i.thriftbooks.com/api/imagehandler/m/4...,2013-04-30T00:00:00,Crossway,208,11.99,John Piper,4.083333,12,5
10048,10048,1000028213,Casablanca (1942),NaN,DVD,https://img.thriftbooks.com/api/images/e/m/43B...,2002-03-07T00:00:00,Warner Home Video,0,4.19,NaN,2.888889,9,3


In [7]:
data_book = df_book.to_numpy()

In [8]:
tfidf_matrix = joblib.load('E:/PBL6/ModelAI/data/TF-IDF/tfidf_matrix_svd4000.pkl')


In [9]:
tfidf_matrix[90]

array([ 1.86646143e-01, -1.14063726e-02, -4.93540994e-03, ...,
        7.21566255e-03,  1.54067022e-04,  9.12000073e-03])

In [10]:
X = df[['book_id',	'user_id',	'age',	'gender_encoded',	'area_encoded']]  # features
y = df['rating']

In [11]:
X_data = X.to_numpy()
row = df_book[df_book['id'] == 248957]
row.to_numpy()

array([[1, 248957, "Harry Potter and the Sorcerer's Stone",
        'beloved first book harry potter series fully illustrated award winning artist jim kay.for first time j.k. rowling beloved harry potter book presented lavishly illustrated full color editions. award winning artist jim kay created 100 stunning illustration making deluxe format perfect gift child introduced series dedicated fans.harry potter never star quidditch team scoring point riding broom far ground. know spell never helped hatch dragon never worn cloak invisibility.all know miserable life dursleys horrible aunt uncle abominable son dudley great big swollen spoiled bully. harry room tiny closet foot stair birthday party eleven years.but change mysterious letter arrives owl messenger letter invitation incredible place harry anyone read find unforgettable.',
        'Paperback',
        'https://i.thriftbooks.com/api/imagehandler/m/96FD86FE12A253FA10F44EBA3CC6E56D4FE738B0.jpeg',
        '1998-09-01T00:00:00', 'Arthur 

In [12]:
# X_feature = []
# for i in range(len(X_data)):
#     book_id = X_data[i][0]
#     find_book = (df_book[df_book['id'] == book_id])
#     find_book = find_book.to_numpy()
#     book_index = find_book[0][0]
#     tf_array = tfidf_matrix[book_index]
#     # print('tfidf_matrix[book_index]',tf_array.shape)
#     # X_data[i] = np.concatenate((X_data[i], tf_array[0]))
#     X_i = np.hstack((X_data[i],tf_array))
#     X_feature.append(X_i)
    



In [13]:
from joblib import dump, load

# Save reshaped_array to a file named "data.joblib"
# dump(X_feature, 'tf_matrix_svd_feature.pkl')



In [14]:

# Load data from the "data.joblib" file
X_feature = load('tf_matrix_svd_feature.pkl')

# Now reshaped_array_loaded contains the data loaded from the "data.joblib" file


In [15]:
X_feature[0][2]

39.0

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_feature, y, test_size=0.2, random_state=42)


In [17]:
X_train[1].shape

(4005,)

In [18]:
y_train

151230    5
95076     4
41712     1
169119    1
63282     5
         ..
119879    4
103694    5
131932    1
146867    1
121958    1
Name: rating, Length: 160097, dtype: int64

In [19]:
# from sklearn.ensemble import GradientBoostingClassifier

# GB = GradientBoostingClassifier()
# # pipeline_gb = Pipeline([
# #     ('tfidf', TfidfVectorizer()),
# #     ('GradientBoostingClassifier', GradientBoostingClassifier())
# # ])
# GB.fit(X_train, y_train)
# y_predGB = GB.predict(X_test)


KeyboardInterrupt: 

In [ ]:
# from sklearn.metrics import mean_squared_error

# # Đánh giá mô hình bằng Mean Squared Error (MSE)
# mseGB = mean_squared_error(y_test, y_predGB)
# print("Mean Squared Error:", mseGB)

## Linear Regression

In [ ]:
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()

# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)



In [ ]:
model_LinearRegression = load('model_LinearRegression.pkl')


In [ ]:
rows_with_userid_3 = df[df['user_id'] == 5635]
rows_with_userid_3

,rating_id,book_id,user_id,rating,age,gender,area,gender_encoded,area_encoded
1263,1263,249355,5635,5,26,Female,south_asian,0,9
49633,49633,250132,5635,5,26,Female,south_asian,0,9
62130,62130,249994,5635,5,26,Female,south_asian,0,9
85449,85449,248592,5635,5,26,Female,south_asian,0,9
90397,90397,246039,5635,2,26,Female,south_asian,0,9
181771,181771,14572273,5635,5,26,Female,south_asian,0,9


In [ ]:
user = X_data[5635][1:]
# X_data_modified = X_data[:, 1:]
book_id_test = [246045]
input = np.concatenate((book_id_test, user))
input

array([246045,   7417,     30,      0,      3], dtype=int64)

In [ ]:
model_LinearRegression.predict(input)

In [ ]:
# dump(model, 'model_LinearRegression.pkl')


Mean Squared Error: 2.388487298461521
Mean Squared Error: 0.9910420985621311


In [ ]:
from sklearn.metrics import mean_squared_error

# Đánh giá mô hình bằng Mean Squared Error (MSE)
# mse = mean_squared_error(y_test, y_pred)
# print("Mean Squared Error:", mse)

In [ ]:
# errors = y_test - y_pred

# # Tính toán RMSE
# rmse = np.sqrt(np.mean(errors**2))
# print("Root Mean Squared Error (RMSE):", rmse)

Root Mean Squared Error (RMSE): 0.9955109736020649


In [ ]:
from sklearn.linear_model import LogisticRegression
modelLogisticRegression = LogisticRegression()

modelLogisticRegression.fit(X_train, y_train)
y_pred = modelLogisticRegression.predict(X_test)

In [ ]:
mseLogisticRegression = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mseLogisticRegression)

Mean Squared Error: 3.744959400374766


In [ ]:
# !pip install auto-sklearn

In [ ]:
# !pip install -U scikit-learn


In [ ]:
# import autosklearn.classification
